<div class="frontmatter text-center">
<h1>Geospatial Data Science</h1>
<h2>Exercise 12: Mobility with scikit-mobility</h2>
<h3>IT University of Copenhagen, Spring 2023</h3>
<h3>Instructors: Jan Leonard Schelhaas</h3>
</div>

In today's exercise we will practice working with the scikit-mobility package and get some hands on experience with how to handle mobility data.

This notebook was developed with inspration from the [scikit-mobility tutorials](https://github.com/scikit-mobility/scikit-mobility/tree/master/examples) and [examples](https://github.com/scikit-mobility/scikit-mobility/tree/master/examples). 

### Working with scikit mobility - useful links

* [Documentation and install instructions](https://scikit-mobility.github.io/scikit-mobility/)
* [GitHub Repository](https://github.com/scikit-mobility/scikit-mobility)
* [Examples and demos](https://github.com/scikit-mobility/scikit-mobility/tree/master/examples)
* [Tutorials](https://github.com/scikit-mobility/tutorials/tree/master/mda_masterbd2020)
* [See also the partly overlapping package *movingpandas*](https://github.com/movingpandas/movingpandas)
## Short Instructions 
The exercise is divided into 2 parts: 
One with flow data and one with trajectory data. 

For each part the work flow is to:

1. **Load the data** (We have done a bit of the pre-processing for you)

2. **Convert it to a scikit-mobility format (i.e. respectively a FlowDataFrame and a TrajDataFrame)** (Already done for the trajectories)

3. **Visualise the data in a way of your own choice (*we recommend using scikit-mobility's build-in support for easy and nice looking plots with folium*)**

4. **Analyse some aspect of the data with a measure of your own choice. You can see all build-in measures [right here](https://scikit-mobility.github.io/scikit-mobility/reference/measures.html).** 
    *Most of the build in methods are for trajectory data, but see if you can come up with something for the flow data yourself.*


## Data
The **flow** data we will be using today is an open data set of origins and destinations of trips on Oslo's city bikes.

A json file with trips from February 2022 is available with this notebook. You can also download more data from Oslo Bysykkel [here](https://oslobysykkel.no/apne-data/historisk).

The **trajectory** data is a dataset of taxi trips in Porto from 2015. Data can be downloaded [here](https://archive.ics.uci.edu/ml/datasets/Taxi+Service+Trajectory+-+Prediction+Challenge,+ECML+PKDD+2015) (train.csv.zip), where you also can read more about the dataset. This data set is very big - we suggest to only import the first x rows (x in the order of 10^4).

Below, we give you an example of the visual outcome for this exercise.

#### Example of output from flow data visualisation (showing the tessellation data)

In [1]:
from IPython.display import IFrame
IFrame(src='results_examples/exercise11_example_tesselation.html', width=900, height=700)

#### Example of output from flow data visualisation (showing the flows)

In [2]:
IFrame(src='results_examples/exercise11_example_flows.html', width=900, height=700)

# Coding starts here :)

In [ ]:
# import libraries needed
import skmob
import geopandas as gpd
import pandas as pd
from skmob.tessellation import tilers
from skmob.utils import plot
import matplotlib.pyplot as plt
import numpy as np
import datetime

tess_style = {'color':'gray', 'fillColor':'gray', 'opacity':0.2}

## Flows

Flow data is also often known as origin-destination data (O-D) and, as implied by the name, describes the origin and destination of a trip - but not the actual trajectory between the two points.

In our dataset we have all the information on the coordinates for origin and destination in the same row, so we need to modify the data structure a little bit before we can convert it to a scikit-mobility dataframe.
The goal is to have a dataframe were each row as the id of the origin and destination location, plus a geodataframe with the location referenced by the origins and destinations.

This geodataframe is usually referred to as the 'tesselation' - but you can still use point geometries as tesselations, as we will be doing here.

In [ ]:
# Load data
bike_trips = pd.read_json('data/oslo_bysykkel_0222.json')

bike_trips.head()

### A bit of data processing...

In [ ]:
# Reformat data

ods = bike_trips[['start_station_id','end_station_id']].copy(deep=True) # You can include more columns if you want to use them in your analysis
tesselation_start = bike_trips[['start_station_name','start_station_id','start_station_latitude','start_station_longitude']].copy(deep=True)
tesselation_end = bike_trips[['end_station_name','end_station_id','end_station_latitude','end_station_longitude']].copy(deep=True)

# Count trips between the same origin and destination
ods['flow'] = 1
flows = ods.groupby(['start_station_id','end_station_id']).flow.count().reset_index()

# Rename columns
tesselation_start.rename(columns={'start_station_name':'station_name','start_station_id':'tile_ID','start_station_latitude':'latitude','start_station_longitude':'longitude'},inplace=True)
tesselation_start.drop_duplicates(inplace=True)
tesselation_end.rename(columns={'end_station_name':'station_name','end_station_id':'tile_ID','end_station_latitude':'latitude','end_station_longitude':'longitude'},inplace=True)
tesselation_end.drop_duplicates(inplace=True)

# Combine and remove duplicates
tesselation = pd.concat([tesselation_start, tesselation_end], ignore_index=True)
tesselation.drop_duplicates(inplace=True)
# In our data all origin locations appear to also be destinations and vice versa, but just to be use we combine both

# Convert to geodataframe
tess = gpd.GeoDataFrame(tesselation, geometry=gpd.points_from_xy(tesselation.longitude, tesselation.latitude), crs='EPSG:4326') # There is no defined CRS in the data, but we can read on Oslo Bysykkels website that coordinates are in WGS 84

In [ ]:
# Load to Flowdataframe 

# Add you code here

### Plot tesselation

### Plot flows

## Trajectories

With trajectory data, we both have location of start and end point, but also of the location *in-between* the origin and destination (depending on the quality of the trajectory data it might represent the actual trajectory or be closer to a 'sample' of the location points defining the trajectory).

In [ ]:
# import data set

df = pd.read_csv('data/Porto_taxi_data_test_partial_trajectories.csv', sep=',')
df.head()

In [ ]:
# process data - each point from the polyline should have its own row
# each point needs lat and lon coordinates in separate columns
rows = []
for i, row in df.iterrows():
    uid, tid = row['TAXI_ID'], row['TRIP_ID']
    call_type = row['CALL_TYPE']
    timestamp = row['TIMESTAMP']
    day_type = row['DAY_TYPE']
    for point in eval(row['POLYLINE']):
        rows.append([uid, tid, call_type, timestamp, day_type, point[1], point[0]])

temp_df = pd.DataFrame(rows, columns=['uid', 'tid', 'call_type', 'datetime', 
                                       'day_type', 'lat', 'lng'])
temp_df['datetime'] = pd.to_datetime(temp_df['datetime'], unit='s')
tdf = skmob.TrajDataFrame(temp_df)
tdf.head()

In [ ]:
# Inspect trajectories data frame:
print(tdf.crs)
print()
print(tdf.parameters)
print()
print(tdf.dtypes)
print()
print("Unique UIDs: ", len(np.unique(tdf.uid)))

In [ ]:
# ADD YOUR CODE HERE

### Plot trajectories

In [ ]:
# ADD YOUR CODE HERE